In [ ]:
"""
Neutral atom image analysis wrapper
"""

import sys
import os
from neutral_atom_image_analysis import ImageAnalysis, ImageAnalysisProjection

import matplotlib.pyplot as plt
import h5py

import numpy as np

camera = 'orca0'
path_closed_shutter = "<path_to_data>"
directory_closed_shutter = os.fsencode(path_closed_shutter)
first = True
image_count = 0
for file in os.listdir(directory_closed_shutter):
    filename = os.fsdecode(file)
    with h5py.File(path_closed_shutter + filename) as f:
        for i in f.get(camera).keys():
            if first:
                metadata = f.get(camera + '/' + i + '/metadata')
                for key in metadata.keys():
                    value = str(metadata[key][()])
                    print(key + ": " + value)
                    if key == "roiwidth":
                        width = int(float(value))
                    if key == "roiheight":
                        height = int(float(value))
                loc_image = np.zeros((height,width))

            data = np.array(f.get(camera + '/' + i + '/data'))

            if(len(data) > 0):
                image_count += 1
                if first:
                    closed_shutter_average = data.astype(np.float64)
                    first = False
                else:
                    closed_shutter_average += data
closed_shutter_average = closed_shutter_average / image_count

path = "<path_to_data>"
directory = os.fsencode(path)
images = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    with h5py.File(path + filename) as f:
        for i in f.get(camera).keys():
            data = np.array(f.get(camera + '/' + i + '/data'))
            if(len(data) > 0):
                images.append(data.copy())

# Only psf_supersample=1 works properly for now
iap = ImageAnalysisProjection(1)
iap.calibrate(images, closed_shutter_average)

parameters = []
for image in images:
    result = iap.reconstruct(image)
    parameters.extend(result)

plt.hist(parameters, int(np.sqrt(len(parameters))))